# Performance Overview:

1. **All loaded data with k=3:**
     
     1.1. LightGBM score =

## 1) Importing and loading

In [1]:
! pip install gpyopt

    100% |████████████████████████████████| 61kB 2.9MB/s ta 0:00:011
    100% |████████████████████████████████| 880kB 22.7MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 23.8MB/s ta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/33/1d/87/dc02440831ba986b1547dd11a7dcd44e893b0527083066d869
  Stored in directory: /tmp/.cache/pip/wheels/97/82/1d/32a361e1ff2b4d9129a60343831dd99cdc74440e2db1c55264
  Stored in directory: /tmp/.cache/pip/wheels/a9/fc/74/3bbd263c43ed98d67343df24cebf0a0ee34afee40d769fda9c
Successfully built gpyopt GPy paramz


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/1-stats-k3"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test_milos.csv', 'train_milos.csv']


In [3]:
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
# import GPyOpt
from imblearn.over_sampling import ADASYN
from collections import Counter
import lightgbm as lgb
from datetime import datetime
from numba import jit

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import GPyOpt

Using TensorFlow backend.


In [4]:
# Read in features from GitHub
train_data = pd.read_csv('../input/1-stats-k3/train_milos.csv')
test_data = pd.read_csv('../input/1-stats-k3/test_milos.csv')
sample_submission = pd.read_csv('../input/1-stats-k3/sample_submission.csv')

print('Training data shape: ', train_data.shape)
print('Testing data shape:  ', test_data.shape)

Training data shape:  (200000, 213)
Testing data shape:   (200000, 212)


In [5]:
train_data.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,sum,min,max,mean,std,skew,kurt,med,mean_dist_3,max_dist_3,min_dist_3
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.0
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,7.567236,0.394340,-3.245596,14.023978,8.530232,7.537606,14.573126,9.333264,-5.696731,15.244013,12.438567,13.290894,17.257883,4.305430,3.019540,10.584400,13.667496,-4.055133,-1.137908,5.532980,5.053874,-7.687740,10.393046,-0.512886,14.774147,11.434250,3.842499,2.187230,5.868899,10.642131,...,-0.831777,19.817094,-0.677967,20.210677,11.640613,-2.799585,11.882933,-1.014064,2.591444,-2.741666,10.085518,0.719109,8.769088,12.756676,-3.983261,8.970274,-10.335043,15.377174,0.746072,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537,1353.489771,-30.444924,39.278867,6.767449,9.662541,-0.179883,2.251779,6.668707,55.981007,84.729509,0.0
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,1.235070,5.500793,5.970253,0.190059,4.639536,2.247908,0.411711,2.557421,6.712612,7.851370,7.996694,5.876254,8.196564,2.847958,0.526893,3.777245,0.285535,5.922210,1.523714,0.783367,2.615942,7.965198,2.159891,2.587830,4.322325,0.541614,5.179559,3.119978,2.249730,4.278903,...,5.378008,8.674171,5.966674,7.136427,2.892167,7.513939,2.628895,8.579810,2.798956,5.261243,1.371862,8.963434,4.474924,9.318280,4.725167,3.189759,11.574708,3.944604,0.976348,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015,80.309428,9.054951,5.642146,0.401547,0.565214,0.392823,1.456123,0.402653,2.278199,3.389279,0.0
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,3.970500,-20.731300,-26.095000,13.434600,-6.011100,1.013300,13.076900,0.635100,-33.380200,-10.664200,-12.402500,-5.432200,-10.089000,-5.322500,1.209800,-0.678400,12.720000,-24.243100,-6.166800,2.089600,-4.787200,-34.798400,2.140600,-8.986100,1.508500,9.816900,-16.513600,-8.095100,-1.183400,-6.337100,...,-22.479300,-11.453300,-22.748700,-2.995300,3.241500,-29.116500,4.952100,-29.273400,-7.856100,-22.037400,5.416500,-26.001100,-4.808200,-18.489700,-22.583300,-3.022300,-47.753600,4.412300,-2.554300,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800,992.663500,-90.252500,22.458300,4.963318,7.369270,-3.329985,-0.451972,5.001200,46.700465,71.03

In [6]:
# Convert to numpy arrays
original_features  = train_data.drop(columns=['ID_code', 'target'])
testing_features = test_data.drop(columns=['ID_code'])

# Sklearn wants the labels as one-dimensional vectors
original_targets  = train_data.filter(items=['target'])

## 2) Augmentation

RESULT: Simple oned showed OK, while SMOTE and ADASYN where bad!!

TO-DO: GANs or (Variational) AutoEncoder

### 2.1 [Ole](https://www.kaggle.com/omgrodas/lightgbm-with-data-augmentation/log)

In [7]:
## Inspiration from
#https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment
def augment(train, target, num_n=1, num_p=2):
    newtrain=[train]
    newtarget=[target]
    
    n=train[target==0]
    z=target[target==0]
    for i in range(num_n):
        random_state = np.random.permutation(len(n))
        newtrain.append(n.apply(lambda x:x.values.take(random_state)))
        newtarget.append(z.apply(lambda x:x.values.take(random_state)))
    
    for i in range(num_p):
        p=train[target>0]
        o=target[target>0]
        random_state = np.random.permutation(len(p))
        newtrain.append(p.apply(lambda x:x.values.take(random_state)))
        newtarget.append(o.apply(lambda x:x.values.take(random_state)))
    return pd.concat(newtrain), pd.concat(newtarget)

### 2.2 [Nanashi](https://www.kaggle.com/jesucristo/santander-magic-lgb-0-901)

Result is the same as from Ole, but this function is faster!!!

In [8]:
# @jit
def augment_fast(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        mask = mask[:,0]
        x1 = x[mask,:].copy()
        ids = np.arange(x1.shape[0])
        for p in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,p] = x1[ids][:,p]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        mask = mask[:,0]
        x1 = x[mask, :].copy()
        ids = np.arange(x1.shape[0])
        for n in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,n] = x1[ids][:,n]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
#     print(y.shape)
#     print(ys.shape)
#     print(yn.shape)
    y = np.concatenate([y[:,0],ys,yn])
    return x,y

## 3) Defining training datasets

### 3.1 All non-augmented data

In [9]:
training_features = original_features
training_targets = original_targets

### 3.2 Simple augmentation of training data

In [15]:
# done in the model loop:
simple_aug = False

### 3.3 Only FE-features

In [11]:
# training_features = original_features[original_features.columns[200:]]
# training_targets = original_targets
# testing_features = testing_features[testing_features.columns[200:]]

### 3.X Size-check:

In [12]:
print('Training feature shape: ', training_features.shape)
print('Training targe shape:  ', training_targets.shape)
print('Testing targe shape:  ', testing_features.shape)

Training feature shape:  (200000, 211)
Training targe shape:   (200000, 1)
Testing targe shape:   (200000, 211)


## 4) First-level models

I will try here first LightGMB, as it showed best in practice by others (and I love it!), then I have GaussianNB, Logistic Regression and Fast Keras. SVM could be a good attempt and maybe some different ANN...

### 4.1 LightGBM

In [ ]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.1, #0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': -1,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1
}

In [ ]:
%%time
result=np.zeros(testing_features.shape[0])

skf = StratifiedKFold(n_splits=5, random_state=None)
for counter,(train_index, valid_index) in enumerate(skf.split(training_features, training_targets),1):
    print (counter)
    
    X_train, y_train = training_features.iloc[train_index], training_targets.iloc[train_index]
    X_valid, y_valid = training_features.iloc[valid_index], training_targets.iloc[valid_index]
    
    if simple_aug:
        X_train, y_train = augment_fast(X_train.values, y_train.values)
        X_train = pd.DataFrame(X_train)
        print("Augmenation done!")
    
    # LightGBM data
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_val = lgb.Dataset(X_valid, label=y_valid)
    
    #Training
    model_lgb = lgb.train(param, lgb_train, 1000, valid_sets = [lgb_train, lgb_val], verbose_eval=20, early_stopping_rounds = 50)
    result += model_lgb.predict(testing_features, num_iteration=model_lgb.best_iteration)

In [ ]:
submission_lgb = sample_submission.copy()
submission_lgb['target'] = result/counter
filename="{:%Y-%m-%d_%H_%M}_lgb.csv".format(datetime.now())
submission_lgb.to_csv(filename, index=False)

### 4.2 Fast Keras

In [ ]:
# scaler = StandardScaler()
scaler = MinMaxScaler()

scaler.fit(training_features)
training_features_scaled = pd.DataFrame(scaler.transform(training_features))
testing_features_scaled = pd.DataFrame(scaler.transform(testing_features))

In [ ]:
# Build neural network
def create_model():
    model = Sequential()
    model.add(Dense(2**12, activation='relu', input_dim=training_features.shape[1]))
    model.add(Dropout(rate=0.5))
    model.add(Dense(2**8, activation='relu'))
    model.add(Dropout(rate=0.4))
    model.add(Dense(2**6, activation='linear'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(2**4, activation='linear'))
    # model.add(Dropout(rate=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Defining optimizer:
    my_opt = optimizers.Adam(lr=0.0001, beta_1=0.9995, beta_2=0.99999, epsilon=0.01, decay=1e-6, amsgrad=True)
    # my_opt = optimizers.SGD(lr=0.0001, momentum=0.5, decay=1e-6, nesterov=True)

    # Compile model
    model.compile(optimizer=my_opt,
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    
    return model

In [ ]:
model_ann = create_model()

result_ann=np.zeros(testing_features.shape[0])

skf = StratifiedKFold(n_splits=2, random_state=None)
for counter,(train_index, valid_index) in enumerate(skf.split(training_features_scaled, training_targets),1):
    print(counter)
    
    X_train, y_train = training_features_scaled.iloc[train_index], training_targets.iloc[train_index]
    X_valid, y_valid = training_features_scaled.iloc[valid_index], training_targets.iloc[valid_index]
    
    if simple_aug:
        X_train, y_train = augment_fast(X_train.values, y_train.values)
        X_train = pd.DataFrame(X_train)
        print("Augmenation done!")
    
    #Training
    ### training
    es = EarlyStopping(monitor='val_binary_accuracy', mode='auto', verbose=0, patience=400)
    cp = ModelCheckpoint(filepath='/best_weights_'+str(counter)+'_.hdf5', verbose=1, save_best_only=True)
    model_ann.fit(X_train, y_train, batch_size=2**8, epochs=10000, validation_data=(X_valid, y_valid), callbacks=[es, cp], verbose=1)
    
    model_best = create_model()
    model_best.load_weights('/best_weights_'+str(counter)+'_.hdf5')
    result_ann += model_best.predict(testing_features_scaled)[:,0]

In [ ]:
submission_ann = sample_submission.copy()
submission_ann['target'] = result/counter
filename="{:%Y-%m-%d_%H_%M}_ann.csv".format(datetime.now())
submission_ann.to_csv(filename, index=False)

### 4.3 GaussianNB

In [16]:
%%time
result_gnb=np.zeros(testing_features.shape[0])

skf = StratifiedKFold(n_splits=5, random_state=None)
for counter,(train_index, valid_index) in enumerate(skf.split(training_features, training_targets),1):
    print (counter)
    
    X_train, y_train = training_features.iloc[train_index], training_targets.iloc[train_index]
    X_valid, y_valid = training_features.iloc[valid_index], training_targets.iloc[valid_index]
    
    if simple_aug:
        X_train, y_train = augment_fast(X_train.values, y_train.values)
        X_train = pd.DataFrame(X_train)
        print("Augmenation done!")
    
    skf_inner = StratifiedKFold(n_splits=4, random_state=None)

    # Optimizing
    # Score. Optimizer will try to find minimum, so we will add a "-" sign.
    def ml_fun(parameters):
        parameters = parameters[0]
        score = -cross_val_score(GaussianNB(priors=None, var_smoothing = parameters[0]),
            X_train, y_train, cv=skf_inner.split(X_train, y_train), scoring='roc_auc', n_jobs=-1).mean()
        score = np.array(score)
        return score

    # Bounds (NOTE: define continuous variables first, then discrete!)
    bounds = [
                {'name': 'var_smoothing', 'type': 'continuous', 'domain': (1e-11, 1e-7)}
             ]

    optimizer = GPyOpt.methods.BayesianOptimization(f=ml_fun, domain=bounds,
                                                acquisition_type ='MPI',
                                                exact_eval=True)

    optimizer.run_optimization(1e3, 1e4, verbosity=True)
    
    #Training & validation
    gnb = GaussianNB(priors=None, var_smoothing = optimizer.x_opt[0]).fit(X_train, y_train)
    GaussNB_val_proba = gnb.predict_proba(X_valid)
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_valid, GaussNB_val_proba[:, 1])
    print("GaussianNB validation {}: {:.5f}".format(counter, auc(fpr_keras, tpr_keras)))
    
    result_gnb += gnb.predict_proba(testing_features)[:, 1]

1
num acquisition: 1, time elapsed: 1.60s
num acquisition: 2, time elapsed: 3.13s
[0.00891455 0.94694265 0.00204979 ... 0.01765948 0.00538334 0.02424887]
GaussianNB validation 1: 0.86665
[0.99989965 0.99998667 0.99999873 ... 0.99985888 0.99839121 0.98164244]
2
num acquisition: 1, time elapsed: 1.69s
num acquisition: 2, time elapsed: 3.22s
[0.05945756 0.48423989 0.01813614 ... 0.84185809 0.31472265 0.10434182]
GaussianNB validation 2: 0.86062
[0.99989054 0.99998578 0.99999862 ... 0.99984461 0.99873736 0.9869225 ]
3
num acquisition: 1, time elapsed: 1.56s
num acquisition: 2, time elapsed: 3.06s


KeyboardInterrupt: 

In [ ]:
submission_gnb = sample_submission.copy()
submission_gnb['target'] = result_gnb/counter
filename="{:%Y-%m-%d_%H_%M}_gbn.csv".format(datetime.now())
submission_gnb.to_csv(filename, index=False)

### 4.4 Logistic Regression

In [19]:
%%time
result_LogR=np.zeros(testing_features.shape[0])

skf = StratifiedKFold(n_splits=5, random_state=None)
for counter,(train_index, valid_index) in enumerate(skf.split(training_features, training_targets),1):
    print (counter)
    
    X_train, y_train = training_features.iloc[train_index], training_targets.iloc[train_index]
    X_valid, y_valid = training_features.iloc[valid_index], training_targets.iloc[valid_index]
    
    if simple_aug:
        X_train, y_train = augment_fast(X_train.values, y_train.values)
        X_train = pd.DataFrame(X_train)
        print("Augmenation done!")
    
    skf_inner = StratifiedKFold(n_splits=4, random_state=None)

    # Optimizing
    # Score. Optimizer will try to find minimum, so we will add a "-" sign.
    def ml_fun(parameters):
        parameters = parameters[0]
        score = -cross_val_score(LogisticRegression(penalty='l2', dual=False, solver='newton-cg', multi_class='auto',
                           tol = parameters[0],
                           C = parameters[1], 
                           max_iter = parameters[2],
                           n_jobs=-1),
            X_train, y_train, cv=skf_inner.split(X_train, y_train), scoring='roc_auc', n_jobs=-1).mean()
        score = np.array(score)
        return score

    # Bounds (NOTE: define continuous variables first, then discrete!)
    bounds = [
            {'name': 'tol', 'type': 'continuous', 'domain': (1e-5, 1e-3)},
            {'name': 'C', 'type': 'continuous', 'domain': (10**-2, 10**2)},
            {'name': 'max_iter', 'type': 'discrete', 'domain': (10**1, 10**3)}
         ]

    optimizer = GPyOpt.methods.BayesianOptimization(f=ml_fun, domain=bounds,
                                                acquisition_type ='MPI',
                                                exact_eval=True)

    optimizer.run_optimization(1e3, 1e4, verbosity=True)
    
    #Training & validation
    LogR = LogisticRegression(penalty='l2', dual=False, solver='newton-cg', multi_class='auto',
                              tol = optimizer.x_opt[0],
                              C = optimizer.x_opt[1],
                              max_iter = optimizer.x_opt[2],
                              n_jobs=-1).fit(X_train, y_train)
    LogR_val_proba = LogR.predict_proba(X_valid)
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_valid, LogR_val_proba[:, 1])
    print("LogRegression validation {}: {:.5f}".format(counter, auc(fpr_keras, tpr_keras)))
    
    result_LogR += LogR.predict_proba(testing_features)[:, 1]

1


KeyboardInterrupt: 

In [ ]:
submission_LogR = sample_submission.copy()
submission_LogR['target'] = result_LogR/counter
filename="{:%Y-%m-%d_%H_%M}_gbn.csv".format(datetime.now())
submission_LogR.to_csv(filename, index=False)